# **Лабораторна робота 6: Пошук аномалій та вирішення задачі *anomaly detection* за допомогою бібліотек `scikit-learn`та `PyTorch`**
**Всі завдання виконуються індивідуально. Використання запозиченого коду буде оцінюватись в 0 балів.**

**Лабораторні роботи де в коді буде використаня КИРИЛИЦІ будуть оцінюватись в 20 балів.**

### Мета роботи:
Ознайомитися з основними методами виявлення аномалій, навчитися використовувати бібліотеки `scikit-learn` та `PyTorch` для реалізації алгоритмів пошуку аномалій, проаналізувати ефективність різних методів на реальних наборах даних з Kaggle.


### Опис завдання:

1. **Постановка задачі**:
   Використовуючи один із доступних наборів даних Kaggle (наприклад, *Credit Card Fraud Detection*, *Network Intrusion*, або інші), вам потрібно розв'язати задачу виявлення аномалій. Основна мета — ідентифікувати аномальні записи серед нормальних. Вибраний набір даних повинен містити мітки аномалій для перевірки результатів.

2. **Етапи виконання завдання**:
   - Завантажте та підготуйте набір даних.
   - Проведіть попередню обробку даних (масштабування, заповнення пропущених значень, видалення нерелевантних ознак).
   - Використайте різні методи виявлення аномалій:
     - **Методи з бібліотеки scikit-learn**:
       - Isolation Forest
       - One-Class SVM
       - Local Outlier Factor (LOF)
     - **Методи з використанням PyTorch**:
       - Автоенкодери для виявлення аномалій.
   - Порівняйте отримані результати, обчисліть метрики якості (Precision, Recall, F1-Score).
   - Оцініть, який метод найкраще підходить для вирішення задачі на вашому наборі даних.

### Покрокова інструкція

1. **Підготовка середовища**:
   - Встановіть необхідні бібліотеки:
     ```
     pip install scikit-learn torch pandas numpy matplotlib
     ```

2. **Вибір набору даних з Kaggle**:
   Зареєструйтесь на Kaggle та оберіть один із наборів даних для виявлення аномалій. Наприклад:
   - [Credit Card Fraud Detection](https://www.kaggle.com/mlg-ulb/creditcardfraud)
   - [Network Intrusion Detection](https://www.kaggle.com/xyuanh/benchmarking-datasets)

3. **Попередня обробка даних**:
   - Завантажте дані та проведіть їхню початкову обробку.
   - Масштабуйте ознаки за допомогою `StandardScaler` або `MinMaxScaler`.
   - Розділіть дані на навчальну і тестову вибірки.

4. **Методи з бібліотеки `scikit-learn`**:

   - **Isolation Forest**:
     ```
     from sklearn.ensemble import IsolationForest
     ```

   - **One-Class SVM**:
     ```
     from sklearn.svm import OneClassSVM
     ```

   - **Local Outlier Factor**:
     ```
     from sklearn.neighbors import LocalOutlierFactor
     ```

5. **Методи на основі нейронних мереж (PyTorch)**:

   Використайте автоенкодер для пошуку аномалій. Побудуйте нейронну мережу з енкодером і декодером. Під час навчання порівняйте відновлені дані з вхідними та обчисліть помилку. Записи з великою помилкою можуть бути аномаліями.

   - **Реалізація автоенкодера**:
     ```
     import torch
     import torch.nn as nn
     import torch.optim as optim
     ```

6. **Оцінка результатів**:
   Використовуйте метрики оцінки якості:
   - `Precision`, `Recall`, `F1-score`
   ```
   from sklearn.metrics import classification_report
   ```

7. **Звіт**:
   - Поясніть, який метод дав найкращі результати.
   - Проаналізуйте, чому деякі методи працюють краще на вашому наборі даних.
   - Оцініть можливості використання глибоких нейронних мереж (автоенкодерів) для вирішення задачі.


### Результати, які необхідно надати:
1. Код рішення у вигляді Jupyter Notebook з аналізом результатів та поясненнями.


### Дедлайн:
[23 жовтня 23:59]


### Корисні ресурси:
- [Документація PyTorch](https://pytorch.org/docs/stable/index.html)
- [Документація scikit-learn](https://scikit-learn.org/stable/documentation.html)
- [Kaggle Datasets](https://www.kaggle.com/datasets)

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score
import torch
import torch.nn as nn
import torch.optim as optim


In [2]:
df = pd.read_csv('creditcard.csv')

print(df.head())

df.info()

print(df.isnull().sum())


   Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.108300  0.005274 -0.190321 -1.175575  0.647376   
4 -0.270533  0.817739  ... -0.009431  0.798278 -0.137458  0.141267 -0.206010   

        V26       V27       V28 

In [3]:
scaler = StandardScaler()
features = df.drop(columns=['Class'])
labels = df['Class']

scaled_features = scaler.fit_transform(features)

X_train, X_test, y_train, y_test = train_test_split(scaled_features, labels, test_size=0.2, random_state=42)


In [4]:
iso_forest = IsolationForest(contamination=0.01, random_state=42)
iso_preds = iso_forest.fit_predict(X_test)
iso_preds = np.where(iso_preds == 1, 0, 1)

print("Isolation Forest:")
print(classification_report(y_test, iso_preds))


Isolation Forest:
              precision    recall  f1-score   support

           0       1.00      0.99      1.00     56864
           1       0.11      0.61      0.18        98

    accuracy                           0.99     56962
   macro avg       0.55      0.80      0.59     56962
weighted avg       1.00      0.99      0.99     56962



In [5]:
oc_svm = OneClassSVM(nu=0.01, kernel="rbf", gamma=0.1)
oc_svm.fit(X_train)
svm_preds = oc_svm.predict(X_test)
svm_preds = np.where(svm_preds == 1, 0, 1)

print("One-Class SVM:")
print(classification_report(y_test, svm_preds))


One-Class SVM:
              precision    recall  f1-score   support

           0       1.00      0.95      0.98     56864
           1       0.03      0.77      0.05        98

    accuracy                           0.95     56962
   macro avg       0.51      0.86      0.51     56962
weighted avg       1.00      0.95      0.97     56962



In [7]:
lof = LocalOutlierFactor(n_neighbors=20, contamination=0.01)
lof_preds = lof.fit_predict(X_test)
lof_preds = np.where(lof_preds == 1, 0, 1)

print("Local Outlier Factor:")
print(classification_report(y_test, lof_preds))


Local Outlier Factor:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99     56864
           1       0.01      0.07      0.02        98

    accuracy                           0.99     56962
   macro avg       0.51      0.53      0.51     56962
weighted avg       1.00      0.99      0.99     56962



In [11]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(X_train.shape[1], 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 8)
        )
        self.decoder = nn.Sequential(
            nn.Linear(8, 16),
            nn.ReLU(),
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, X_train.shape[1])
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


In [12]:
X_train_tensor = torch.FloatTensor(X_train)
X_test_tensor = torch.FloatTensor(X_test)

model = Autoencoder()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

epochs = 20
for epoch in range(epochs):
    output = model(X_train_tensor)
    loss = criterion(output, X_train_tensor)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")


Epoch 1/20, Loss: 1.0174
Epoch 2/20, Loss: 1.0075
Epoch 3/20, Loss: 1.0018
Epoch 4/20, Loss: 0.9975
Epoch 5/20, Loss: 0.9929
Epoch 6/20, Loss: 0.9859
Epoch 7/20, Loss: 0.9758
Epoch 8/20, Loss: 0.9631
Epoch 9/20, Loss: 0.9496
Epoch 10/20, Loss: 0.9368
Epoch 11/20, Loss: 0.9239
Epoch 12/20, Loss: 0.9110
Epoch 13/20, Loss: 0.8993
Epoch 14/20, Loss: 0.8895
Epoch 15/20, Loss: 0.8791
Epoch 16/20, Loss: 0.8669
Epoch 17/20, Loss: 0.8551
Epoch 18/20, Loss: 0.8443
Epoch 19/20, Loss: 0.8344
Epoch 20/20, Loss: 0.8257


In [13]:
with torch.no_grad():
    reconstructions = model(X_test_tensor)
    reconstruction_error = torch.mean((X_test_tensor - reconstructions) ** 2, dim=1).numpy()

threshold = np.percentile(reconstruction_error, 95)
autoencoder_preds = (reconstruction_error > threshold).astype(int)

print("Autoencoder Results:")
print(classification_report(y_test, autoencoder_preds))


Autoencoder Results:
              precision    recall  f1-score   support

           0       1.00      0.95      0.97     56864
           1       0.03      0.82      0.05        98

    accuracy                           0.95     56962
   macro avg       0.51      0.88      0.51     56962
weighted avg       1.00      0.95      0.97     56962



In [16]:
results = {
    "Method": ["Isolation Forest", "One-Class SVM", "Local Outlier Factor", "Autoencoder"],
    "Precision": [precision_score(y_test, iso_preds), precision_score(y_test, svm_preds),
                  precision_score(y_test, lof_preds), precision_score(y_test, autoencoder_preds)],
    "Recall": [recall_score(y_test, iso_preds), recall_score(y_test, svm_preds),
               recall_score(y_test, lof_preds), recall_score(y_test, autoencoder_preds)],
    "F1-Score": [f1_score(y_test, iso_preds), f1_score(y_test, svm_preds),
                 f1_score(y_test, lof_preds), f1_score(y_test, autoencoder_preds)]
}

results_df = pd.DataFrame(results)
print(results_df)


                 Method  Precision    Recall  F1-Score
0      Isolation Forest   0.105263  0.612245  0.179641
1         One-Class SVM   0.026634  0.765306  0.051476
2  Local Outlier Factor   0.012281  0.071429  0.020958
3           Autoencoder   0.028080  0.816327  0.054293


---

- `Precision` показує, як точно модель виявляє аномалії серед усіх передбачених аномалій.
- `Recall` вказує, скільки з усіх реальних аномалій модель змогла знайти.
- `F1-Score` є середнім значенням між `Precision` та `Recall`, показуючи загальну ефективність моделі.

На основі `F1-Score`, `Isolation Forest` має найкращий результат (**0.1796**), але всі методи мають дуже низькі показники.

`Autoencoder` має найвищий `Recall` (**0.8163**), що вказує на здатність моделі знаходити більшість аномалій, але її точність залишає бажати кращого.

Таким чином, якщо головною метою є виявлення якомога більшої кількості аномалій, `Autoencoder` буде кращим вибором, оскільки він має високий `Recall`, хоча й з низькою точністю.

Як покращити ефективність:

- Налаштувати параметри моделей.
- Збільшити обсяг даних.
- Поєднувати методи для стабільніших результатів.
- Покращити нейронні мережі, використовуючи складніші архітектури.
- Очищення даних від шумів та пропусків для кращих результатів.